<a href="https://colab.research.google.com/github/hpt09/ResumeParser/blob/master/Provided_Resources/EditedNameExtractionUsingSnorkel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install snorkel
!pip install nltk
!pip install spacy

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
from snorkel.labeling import labeling_function
import nltk
import spacy

In [15]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
df_dev = pd.read_csv(r"df-dev.csv")

In [12]:
df_dev.head(5)

,content
0,Afreen Jamadar. Active member of IIIT Committe...
1,Alok Khandai. Operational Analyst (SQL DBA) En...
2,Anvitha Rao. Automation developer. . - Email m...
3,arjun ks. Senior Program coordinator - oracle ...
4,"Arun Elumalai. QA Tester. . Chennai, Tamil Nad..."


In [16]:
from nltk import sent_tokenize as st

df_token_sentence = pd.DataFrame(columns=None)

df_token_sentence = df_dev.apply(lambda row: st(row['content']), axis=1)

In [17]:
df_token_sentence.head(2)

0    [Afreen Jamadar., Active member of IIIT Commit...
1    [Alok Khandai., Operational Analyst (SQL DBA) ...
dtype: object

In [19]:
df_token_sentence.to_csv(r"df_token_sentence.csv", header=['content'], index = False)

In [20]:
df_ts = pd.read_csv(r"df_token_sentence.csv")

In [21]:
df_ts.head(1)

,content
0,"['Afreen Jamadar.', 'Active member of IIIT Com..."


In [22]:
keep = False
df = df_ts
column = 'content'
sep = ','
indexes = list()
new_values = list()
df = df.dropna(subset=[column])
for i, presplit in enumerate(df[column].astype(str)):
    values = presplit.split(sep)
    if keep and len(values) > 1:
        indexes.append(i)
        new_values.append(presplit)
    for value in values:
        indexes.append(i)
        new_values.append(value)
new_df = df.iloc[indexes, :].copy()
new_df[column] = new_values
print(new_df)

                                               content
0                                   ['Afreen Jamadar.'
0     'Active member of IIIT Committee in Third year.'
0                                                  '.'
0                                              'Sangli
0     Maharashtra - Email me on Indeed: indeed.com/...
..                                                 ...
665                                                '.'
665                                'ACADEMIC PROJECT.'
665   'Project name:  A Case Study Undertaken to Kn...
665                                Anti-Oxidants and.'
665   'Liver Tests in the Etiology of Turner Syndro...

[86279 rows x 1 columns]


In [23]:
new_df.to_csv(r"tokenized_sentences.csv", index = False)

In [24]:
df_spacy = pd.read_csv(r"tokenized_sentences.csv")

In [25]:
df_spacy.head(3)

,content
0,['Afreen Jamadar.'
1,'Active member of IIIT Committee in Third year.'
2,'.'


In [26]:
len(df_spacy['content'])

86272

In [27]:
remove_punct = df_spacy['content'][2]

In [28]:
df_spacy = df_spacy[df_spacy.content!=remove_punct]

In [29]:
print(df_spacy.head(5))
print(len(df_spacy))

                                             content
0                                 ['Afreen Jamadar.'
1   'Active member of IIIT Committee in Third year.'
3                                            'Sangli
4   Maharashtra - Email me on Indeed: indeed.com/...
6                        'I wish to use my knowledge
69502


In [30]:
df_spacy.to_csv(r"1st_level_cleaned_sentences.csv", index=False, header=['content'])

In [31]:
df_spacy = pd.read_csv("1st_level_cleaned_sentences.csv")
df_spacy.head(10)

,content
0,['Afreen Jamadar.'
1,'Active member of IIIT Committee in Third year.'
2,'Sangli
3,Maharashtra - Email me on Indeed: indeed.com/...
4,'I wish to use my knowledge
5,skills and conceptual understanding to create...
6,'environments and work consistently achieving...
7,'and work to excellence in my work.. .'
8,'WORK EXPERIENCE.'
9,'Active member of IIIT Committee in Third year.'


In [35]:
def clean_text(text):
    text = str(text) 
    text = text.replace("'", "")
    text = text.replace("[", "")
    text = text.replace("]","")
    text = text.replace(".","")
    return text

df_spacy['content']= df_spacy['content'].apply(clean_text)

In [36]:
df_spacy.head(10)

,content
0,Afreen Jamadar
1,Active member of IIIT Committee in Third year
2,Sangli
3,Maharashtra - Email me on Indeed: indeedcom/r...
4,I wish to use my knowledge
5,skills and conceptual understanding to create...
6,environments and work consistently achieving ...
7,and work to excellence in my work
8,WORK EXPERIENCE
9,Active member of IIIT Committee in Third year


In [37]:
df_spacy.to_csv("cleaned_sentences.csv", index=False, header=['content'])

### Name Extraction using Snorkel

In [38]:
import pandas as pd
from snorkel.labeling import labeling_function
import nltk

import spacy
nlp = spacy.load("en_core_web_sm")

In [39]:
df_spacy = pd.read_csv(r"cleaned_sentences.csv")

In [ ]:
# from nltk import sent_tokenize as st

# temp = df_spacy['content'][0]

# nlp = spacy.load("en_core_web_sm")
# text = nlp(temp)

# for ent in text.ents:
#     print(ent.text, ent.label_)

In [40]:
ABSTAIN = -1
POSITIVE = 1
NEGATIVE = 0

In [ ]:
# @labeling_function()
# def lf_start_of_resume(x):
#     print(x)
#     return POSITIVE if len(x)<4 else NEGATIVE
        


# @labeling_function()
# def lf_start_of_resume1(x):
#     print(x)
#     return POSITIVE if len(x)<4 else NEGATIVE

    
# @labeling_function()
# def lf_start_of_resume2(x):
#     print(x)
#     return POSITIVE if len(x)<4 else NEGATIVE

In [41]:
@labeling_function()
def lf_name_extraction(x):
    x = x.to_string()
    places_list=['Karnataka', 'Bangalore', 'Tamil Nadu', 'Kanpur','Noida', 'Gujarat', 'Haryana', 'Bengaluru', 'Uttar Pradesh', 'Kolhapur', 'Kerala' ,'Maharashtra']
    
    ners = nlp(x)
    for ent in ners.ents:
        if(ent.label_ == "PERSON"):
            for place in places_list:
                if(x.find(place)!=-1):
#                     print("negative:", "x:", x, "place:", place)
                    return NEGATIVE
            if(len(x)<20 or len(x)>35):
#                 print("negative: length", len(x), x)
                return NEGATIVE
            else:
#                 print("positive:", x, len(x))
                return POSITIVE
        elif(ent.label_ !="PERSON"):
            return NEGATIVE
#     print("abstain:", x, len(x))
    return ABSTAIN

@labeling_function()
def lf_temp1(x):
    return ABSTAIN
    
@labeling_function()
def lf_temp2(x):
    return ABSTAIN

In [50]:
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel

In [43]:
lfs = [lf_name_extraction,lf_temp1,lf_temp2]

In [ ]:
# lfs = [lf_start_of_resume,lf_start_of_resume1,lf_start_of_resume2]

In [47]:
applier = PandasLFApplier(lfs)

In [48]:
L_train = applier.apply(df_spacy)

100%|██████████| 69481/69481 [11:31<00:00, 100.48it/s]


In [ ]:
# Y_dev = pd.read_csv(r"Y_dev.csv").as_matrix()

In [ ]:
# LFAnalysis(L_dev,lfs).lf_summary(Y_dev)

In [51]:
label_model = LabelModel(cardinality=3, verbose=True)

In [52]:
label_model.fit(L_train, n_epochs=500, log_freq=50, seed=123)

In [53]:
df_spacy["label"] = label_model.predict(L=L_train, tie_break_policy="abstain")

In [54]:
df_spacy["label"]

0        1
1        0
2       -1
3        0
4       -1
        ..
69476   -1
69477   -1
69478   -1
69479    1
69480   -1
Name: label, Length: 69481, dtype: int64

In [55]:
df_spacy.to_csv(path_or_buf = 'snorkel_labeled.csv',index=False)
df_spacy = df_spacy[df_spacy.label != ABSTAIN]

### Training a Classifier

In [56]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [57]:
train_text = df_spacy.content.tolist()
cv = CountVectorizer(ngram_range=(1,2)).fit(train_text)
X_train = cv.transform(train_text)

In [58]:
clf = LogisticRegression(solver="newton-cg")
clf = clf.fit(X=X_train, y=df_spacy.label.values)

In [59]:
test = pd.read_csv(r'test_files\test_tokenized_sentences.csv')

FileNotFoundError: ignored

In [ ]:
test

,content
0,Rahul Kumar
1,Fresher
2,Bengaluru
3,Karnataka - Email me on Indeed: indeedcom/r/R...
4,Looking for a challenging carrier to enhance ...
...,...
2476,in final
2477,Aff
2478,To CCS
2479,University


In [ ]:
test_text = test.content.tolist()
T_train = cv.transform(test_text)

In [ ]:
test['prediction'] = clf.predict(X=T_train)

In [ ]:
test['prediction']

0       1
1       0
2       0
3       0
4       0
       ..
2476    0
2477    0
2478    0
2479    0
2480    0
Name: prediction, Length: 2481, dtype: int32

In [ ]:
test.to_csv('test_files\snorkel_test_predictions.csv', index=False)

### Calculating Accuracy

In [ ]:
df_cleaned = pd.read_csv(r"cleaned_dataset.csv")

In [ ]:
df_label = df_cleaned['label']

In [ ]:
df_label.head(5)

0    Afreen Jamadar
1      Alok Khandai
2       Anvitha Rao
3          arjun ks
4     Arun Elumalai
Name: label, dtype: object

In [ ]:
df_positive = df_spacy[df_spacy['label']==1]

In [ ]:
df_positive.head(5)

,content,label
0,Afreen Jamadar,1
46,Alok Khandai,1
243,Anvitha Rao,1
524,Ashalata Bisoyi,1
578,Ashok Kunam,1


In [ ]:
count = 0
for name in df_label:
#     name = name.to_string()
    if(df_positive.content.to_string().find(name)!=-1):
        count +=1
    else:
        print(name)

arjun ks
Arun Elumalai
Ayesha B
Darshan G.
Govardhana K
Ijas Nizamuddin
Karthihayini C
Kavitha K
Kavya U.
kimaya sonawane
Mohamed Ameen
Nitin Tr
Pradeeba V
PRASHANTH BADALA
Pratibha P
Prem Koshti
Ram Edupuganti
Ramesh HP
Ramya. P
R Arunravi
Sai Dhir
Sameer Kujur
Shabnam Saba
Shaheen Unissa
Shreyanshu Gupta
Soumya Balan
Syam Devendla
Vamsi krishna
VARUN AHLUWALIA
Vijayalakshmi Govindarajan
Yasothai Jayaramachandran
Yathishwaran P
amarjyot sodhi
Sameer Kujur
Abdul B
Ramesh chokkala
Ganesh AlalaSundaram
Bangalore Tavarekere
Tanmoy Maity
Chhaya Prabhale
Karthik G V
Soumya Balan
Akshay Dubey
B. Gokul
Anand S
Priyesh Dubey
Laya A
Vishwanath P
Ramakrishna Rao
Keshav Dhawale
Gunjan Nayyar
Rupesh Reddy
Puneeth R
Deepika S
Georgia Institute of Technology
Shaik Tazuddin
Angad Waghmare
Tapan kumar Nayak
Palani S
Mayank Shukla
Sridevi H
Raktim Podder
Pavithra M
shrikant desai
K. Siddharth
Venkateswara D
Siddharth Choudhary
Pradeep Kumar
Ajay Gupta
sneh jain
Rahul singh
sneh jain
Ram Dubey
MUKESH SH

In [ ]:
print(count) #Only 449 names were identified out of 667 names

449


In [ ]:
print("Accuracy:", count/667*100)

Accuracy: 67.31634182908546


### Utilities -------------------------------------------------------------------------------------------------------------

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("TIBCO Administrator 560")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [ ]:
df_abstain_names = pd.read_csv('abstain_names.csv')

In [ ]:
df_abstain_names.head(5)

In [ ]:
identified_count = 0
total_names = df_abstain_names.shape
for name in df_abstain_names['names']:
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(name)
    for ent in doc.ents:
        identified_count +=1
        print(ent.text, ent.start_char, ent.end_char, ent.label_)
print("No.of Identified Names: ", identified_count)
print("No.of unidentified Names: ", total_names-identified_count)

### -------------------------------------------------------------------------------------------------------------